# Agent: Memory

이 튜토리얼을 통해 다음을 배울 수 있다:

- LLM의 상태 비저장(stateless) 특성을 이해한다.
- 대화 컨텍스트를 수동으로 관리하는 기본 메모리를 구현한다.
- mem0 라이브러리를 활용한 지능형 장기 메모리를 구축한다.
- 실전 고객 지원 에이전트에 메모리를 적용한다.

## 1. 환경 설정

### 1.1 필요한 라이브러리 설치

In [ ]:
# 고급 메모리 관리 (mem0)
%pip install -q mem0ai

### 1.2 API 키 설정

In [1]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수를 로드한다
load_dotenv()

# API 키 확인 (선택사항)
api_key = os.getenv('OPENAI_API_KEY')

### 1.3 OpenAI 클라이언트 초기화

In [4]:
from openai import OpenAI

MODEL = "gpt-4o-mini"

# OpenAI 클라이언트를 생성한다
client = OpenAI()

## 2. 기본 메모리: 대화 기록 수동 관리

### 2.1 메모리 없는 대화의 문제점
먼저 메모리가 없을 때 어떤 문제가 발생하는지 확인해본다.

In [5]:
def ask_joke_without_memory():
    """메모리 없이 농담을 요청하는 함수다."""
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "user", "content": "프로그래밍에 대한 농담을 해줘."},
        ],
    )
    return response.choices[0].message.content

def ask_followup_without_memory():
    """메모리 없이 후속 질문을 하는 함수다."""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": "내 이전 질문은 무엇이었나?"},
        ],
    )
    return response.choices[0].message.content

# 실행 예제
joke_response = ask_joke_without_memory()
print("=== 첫 번째 질문 (농담) ===")
print(joke_response)

followup_response = ask_followup_without_memory()
print("\n=== 두 번째 질문 (메모리 없음) ===")
print(followup_response)
# 출력 예상: "죄송하지만, 저는 이전 대화 내용을 기억할 수 없습니다. 어떤 질문을 하셨는지 다시 알려주시겠어요?"

=== 첫 번째 질문 (농담) ===
물론이죠! 여기 프로그래밍 관련 농담 하나 있습니다:

왜 프로그래머는 바다를 좋아할까요?

왜냐하면 바다에 “버그”가 없거든요! 😄

더 필요하시면 언제든지 말씀해 주세요!

=== 두 번째 질문 (메모리 없음) ===
죄송하지만, 이전 질문의 내용을 기억하거나 저장할 수 없습니다. 새로운 질문이 있으시면 언제든지 말씀해 주세요!


**결과**: AI는 방금 전에 농담을 했다는 사실을 전혀 기억하지 못한다.

### 2.2 대화 기록을 포함한 기본 메모리 구현
이전 대화 내용을 `messages` 배열에 포함하면 AI가 맥락을 이해할 수 있다.

In [6]:
def ask_followup_with_memory(joke_response: str):
    """이전 대화 내용을 포함하여 후속 질문을 하는 함수다."""
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            # 첫 번째 대화를 포함한다
            {"role": "user", "content": "프로그래밍에 대한 농담을 해줘."},
            {"role": "assistant", "content": joke_response},
            # 두 번째 질문을 추가한다
            {"role": "user", "content": "내 이전 질문은 무엇이었나?"},
        ],
    )
    return response.choices[0].message.content

# 실행 예제
memory_response = ask_followup_with_memory(joke_response)
print("\n=== 세 번째 질문 (메모리 사용) ===")
print(memory_response)
# 출력 예상: "당신의 이전 질문은 '프로그래밍에 대한 농담을 해줘.'였습니다."


=== 세 번째 질문 (메모리 사용) ===
당신의 이전 질문은 "프로그래밍에 대한 농담을 해줘."였습니다. 추가로 궁금한 점이나 다른 요청이 있으시면 말씀해 주세요!


### 핵심 개념:

- `messages` 배열에 이전 대화를 순서대로 포함한다.
- `role`은 "user", "assistant", "system" 중 하나다.
- AI는 전체 대화 흐름을 보고 맥락을 파악한다.

### 2.3 다중 턴 대화 관리 클래스
실제 애플리케이션에서는 대화 기록을 체계적으로 관리해야 한다.

In [7]:
class SimpleConversation:
    """기본적인 대화 메모리를 관리하는 클래스다."""
    
    def __init__(self, system_prompt: str = None):
        """
        대화를 초기화한다.
        
        Args:
            system_prompt: AI의 역할과 행동을 정의하는 시스템 메시지다.
        """
        self.messages = []
        
        # 시스템 프롬프트가 있으면 첫 메시지로 추가한다
        if system_prompt:
            self.messages.append({"role": "system", "content": system_prompt})
    
    def add_user_message(self, content: str):
        """사용자 메시지를 대화 기록에 추가한다."""
        self.messages.append({"role": "user", "content": content})
    
    def add_assistant_message(self, content: str):
        """AI 응답을 대화 기록에 추가한다."""
        self.messages.append({"role": "assistant", "content": content})
    
    def get_response(self) -> str:
        """현재 대화 기록을 바탕으로 AI 응답을 생성한다."""
        response = client.chat.completions.create(
            model=MODEL,
            messages=self.messages
        )
        
        assistant_message = response.choices[0].message.content
        
        # AI 응답을 대화 기록에 추가한다
        self.add_assistant_message(assistant_message)
        
        return assistant_message
    
    def chat(self, user_message: str) -> str:
        """
        사용자 메시지를 받아 AI 응답을 반환하는 편의 메서드다.
        
        Args:
            user_message: 사용자의 질문이나 메시지다.
            
        Returns:
            AI의 응답이다.
        """
        self.add_user_message(user_message)
        return self.get_response()
    
    def clear_history(self):
        """대화 기록을 초기화한다 (시스템 메시지는 유지)."""
        system_messages = [msg for msg in self.messages if msg["role"] == "system"]
        self.messages = system_messages

# 사용 예제
conversation = SimpleConversation(
    system_prompt="당신은 친절한 프로그래밍 튜터다. 초보자도 이해하기 쉽게 설명한다."
)

print("=== 다중 턴 대화 예제 ===")
print(f"사용자: 파이썬이 뭐야?")
print(f"AI: {conversation.chat('파이썬이 뭐야?')}\n")

print(f"사용자: 그럼 변수는 뭐야?")
print(f"AI: {conversation.chat('그럼 변수는 뭐야?')}\n")

print(f"사용자: 아까 내가 처음에 뭘 물어봤지?")
print(f"AI: {conversation.chat('아까 내가 처음에 뭘 물어봤지?')}")

=== 다중 턴 대화 예제 ===
사용자: 파이썬이 뭐야?
AI: 파이썬(Python)은 널리 사용되는 고급 프로그래밍 언어입니다. 1991년 귀도 반 로섬(Guido van Rossum)에 의해 처음 만들어졌고, 그 이후로 많은 사람들이 사용하고 있습니다. 파이썬은 다음과 같은 특징을 가지고 있습니다:

1. **간결하고 읽기 쉬운 문법**: 파이썬은 문법이 직관적이어서 다른 언어보다 배우기가 쉽습니다. 이는 초보자들이 프로그래밍을 시작하는 데 큰 도움이 됩니다.

2. **다양한 용도**: 웹 개발, 데이터 분석, 인공지능, 자동화, 게임 개발 등 다양한 분야에서 사용할 수 있습니다.

3. **광범위한 라이브러리와 프레임워크**: 파이썬에는 많은 라이브러리(특정 작업을 쉽게 해주는 코드 모음)가 있어서, 복잡한 작업도 쉽게 처리할 수 있습니다. 예를 들어, 데이터 분석에 많이 사용되는 `pandas`, 머신러닝에 사용되는 `scikit-learn`, 웹 개발에 쓰이는 `Django`와 `Flask` 같은 것들이 있습니다.

4. **큰 커뮤니티**: 파이썬은 많은 사람들이 사용하고 있기 때문에, 문제를 해결하기 위한 자료나 도움을 찾기가 쉽습니다.

5. **플랫폼 독립적**: 파이썬은 Windows, macOS, Linux 등 다양한 운영체제에서 사용할 수 있습니다.

이러한 특징들 덕분에 파이썬은 많은 사람들에게 사랑받는 프로그래밍 언어가 되었습니다. 특히 초보자에게 추천되는 언어 중 하나입니다.

사용자: 그럼 변수는 뭐야?
AI: 변수는 프로그래밍에서 데이터를 저장하는 "상자"와 같은 개념입니다. 쉽게 말해, 변수를 사용하면 데이터를 이름으로 붙여서 그 데이터를 추적하고 사용할 수 있습니다. 예를 들어, 사람의 나이, 이름, 점수 같은 정보를 저장할 수 있습니다.

### 변수의 특징

1. **이름**: 변수는 특정한 이름을 가집니다. 이 이름을 통해 변수에 저장된 데이터에 접근할 수 있습니다. 예를 들어, `age`라는 변수에는 나이를 저장할 

## 3. 고급 메모리: mem0 라이브러리

### 3.1 mem0란?
mem0은 AI 에이전트가 장기간에 걸쳐 일관되고 맥락이 풍부한 대화를 유지할 수 있게 해주는 지능형 메모리 아키텍처다.

#### mem0의 주요 특징
mem0은 2단계 메모리 파이프라인을 가진다:
1.  **추출 단계**: 대화에서 핵심 사실만 포착한다.
2.  **업데이트 단계**: 지식 베이스를 지능적으로 관리한다.

주요 특징은 다음과 같다:
- **선택적 메모리 형성**: 전체 대화가 아닌 중요한 정보만 저장한다.
- **지능적 연산**: 메모리를 추가, 업데이트, 삭제할지 자동으로 결정한다.
- **효율적인 검색**: 벡터 임베딩으로 관련 메모리를 빠르게 찾는다.
- **비용 효율성**: 전체 컨텍스트 방식 대비 토큰 사용량을 90% 절약할 수 있다.

### 3.2 mem0 기본 사용법

mem0은 메모리 저장을 위해 벡터 데이터베이스가 필요하다. Qdrant를 사용하는 예제다.

#### 기본 메모리 추가 및 검색

In [8]:
from mem0 import Memory

# Memory 객체 초기화 (기본 설정)
# 기본적으로 메모리는 인메모리(in-memory)로 저장된다.
m = Memory()

# 대화에서 메모리 추출 및 저장
messages = [
    {"role": "user", "content": "오늘 밤 영화를 보려고 하는데, 추천해 줄 만한 게 있나요?"},
    {"role": "assistant", "content": "스릴러 영화는 어떠세요? 꽤 몰입감이 높을 수 있습니다."},
    {"role": "user", "content": "스릴러 영화는 별로 좋아하지 않지만, 공상 과학 영화는 정말 좋아해요."},
    {"role": "assistant", "content": "알겠습니다! 앞으로 스릴러 추천은 피하고 공상 과학 영화를 제안해 드릴게요."},
]

# 메모리 추가
m.add(
    messages, 
    user_id="user_001", 
    metadata={"category": "movie_recommendations"}
)

# 메모리 검색
related_memories = m.search(
    query="나에 대해 무엇을 알고 있니?", 
    user_id="user_001"
)

print("=== 검색된 메모리 ===")
if related_memories and 'results' in related_memories:
    for memory in related_memories['results']:
        print(f"- {memory['memory']}")

=== 검색된 메모리 ===
- 스릴러 영화는 별로 좋아하지 않음
- 공상 과학 영화를 정말 좋아함


### 3.3 Qdrant 설정으로 고급 메모리 구성
더 강력한 영구 메모리 관리를 위해 Qdrant를 명시적으로 설정한다.

In [9]:
from mem0 import Memory

# Qdrant 설정
config = {
    "vector_store": {
        "provider": "qdrant",
        "config": {
            "host": "localhost",
            "port": 6333
        },
    },
}

# 설정을 사용하여 Memory 초기화
# Qdrant Docker 컨테이너가 실행 중이어야 한다.
try:
    memory = Memory.from_config(config)
    print("mem0이 Qdrant에 성공적으로 연결되었다.")
except Exception as e:
    print(f"mem0 연결 실패: {e}")
    print("Qdrant Docker 컨테이너가 실행 중인지 확인해야 한다.")

mem0이 Qdrant에 성공적으로 연결되었다.


### 3.4 메모리를 활용한 지능형 챗봇
mem0을 사용하여 과거 대화를 기억하는 챗봇을 구현한다.

In [10]:
from openai import OpenAI
from mem0 import Memory

# 클라이언트와 메모리 초기화
openai_client = OpenAI()

try:
    memory = Memory.from_config(config)
except Exception as e:
    print(f"메모리 초기화 실패: {e}")
    memory = None # 에러 발생 시 None으로 설정

def chat_with_memories(message: str, user_id: str = "demo_user") -> str:
    """
    메모리를 활용한 대화 함수다.
    """
    if not memory:
        return "메모리 시스템이 초기화되지 않았다."

    # 1. 관련 메모리를 검색한다
    relevant_memories = memory.search(query=message, user_id=user_id, limit=3)
    
    # 2. 메모리를 텍스트로 변환한다
    memories_str = ""
    if relevant_memories and 'results' in relevant_memories:
        memory_texts = [f"- {entry['memory']}" for entry in relevant_memories['results'] if isinstance(entry, dict) and 'memory' in entry]
        if memory_texts:
            memories_str = "\n".join(memory_texts)
    
    # 3. 메모리를 시스템 프롬프트에 포함하여 AI 응답을 생성한다
    system_prompt = f"""당신은 도움이 되는 AI다. 사용자의 질문과 아래 기억을 바탕으로 답변한다.

사용자 기억:
{memories_str if memories_str else "없음"}
"""
    
    chat_messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": message},
    ]
    
    response = openai_client.chat.completions.create(
        model=MODEL, 
        messages=chat_messages
    )
    
    assistant_response = response.choices[0].message.content
    
    # 4. 대화에서 새로운 메모리를 생성한다
    conversation_for_memory = [
        {"role": "user", "content": message},
        {"role": "assistant", "content": assistant_response}
    ]
    
    memory.add(conversation_for_memory, user_id=user_id)
    
    return assistant_response

# 사용 예제
print("=== 메모리 기반 챗봇 데모 ===")
print(f"사용자: 내 이름은 알렉스야.")
print(f"AI: {chat_with_memories('내 이름은 알렉스야.')}\n")

print(f"사용자: 내 이름이 뭐야?")
print(f"AI: {chat_with_memories('내 이름이 뭐야?')}\n")

print(f"사용자: 나는 파이썬 프로그래밍을 배우고 싶어.")
print(f"AI: {chat_with_memories('나는 파이썬 프로그래밍을 배우고 싶어.')}\n")

print(f"사용자: 내가 배우고 싶어하는 게 뭐였지?")
print(f"AI: {chat_with_memories('내가 배우고 싶어하는 게 뭐였지?')}")

=== 메모리 기반 챗봇 데모 ===
사용자: 내 이름은 알렉스야.
AI: 안녕하세요, 알렉스! 만나서 반가워요. 어떻게 도와드릴까요?

사용자: 내 이름이 뭐야?
AI: 당신의 이름은 알렉스입니다.

사용자: 나는 파이썬 프로그래밍을 배우고 싶어.
AI: 알렉스, 파이썬 프로그래밍을 배우고 싶으시군요! 좋은 선택입니다. 파이썬은 배우기 쉽고 다양한 분야에서 활용할 수 있는 강력한 언어입니다. 다음과 같은 단계로 시작해 보세요:

1. **설치하기**: 먼저, Python 공식 웹사이트(https://www.python.org/downloads/)에서 Python을 다운로드하고 설치하세요.

2. **기본 문법 배우기**: 변수, 데이터 타입, 조건문, 반복문, 함수 등을 배워보세요. 온라인 튜토리얼이나 책을 참고할 수 있습니다. 예를 들어, "Automate the Boring Stuff with Python"이라는 책이 초보자에게 좋습니다.

3. **실습하기**: 코드를 직접 작성해보는 것이 중요합니다. 간단한 프로그램이나 스크립트를 만들어보세요. 이를 통해 당신의 이해도를 높일 수 있습니다.

4. **프로젝트**: 실제 프로젝트를 시작해보세요. 관심 있는 분야의 작은 프로젝트를 기획하고 실천하는 것이 많은 도움이 됩니다.

5. **커뮤니티 참여**: Stack Overflow, GitHub, 또는 파이썬 관련 포럼에 참여해보세요. 다른 사람과 상호작용하며 배우는 것도 큰 도움이 됩니다.

혹시 더 구체적인 질문이나 도움이 필요한 부분이 있다면 말씀해 주세요!

사용자: 내가 배우고 싶어하는 게 뭐였지?
AI: 당신은 파이썬 프로그래밍을 배우고 싶어합니다.


## 4. 실전 예제: 고객 지원 에이전트
mem0을 활용한 고객 지원 시스템 구현 예제다.

In [11]:
from openai import OpenAI
from mem0 import Memory

class CustomerSupportAgent:
    """고객 지원을 위한 메모리 기반 AI 에이전트다."""
    
    def __init__(self):
        """
        메모리 구성과 OpenAI 클라이언트로 에이전트를 초기화한다.
        """
        config = {
            "vector_store": {
                "provider": "qdrant",
                "config": {"host": "localhost", "port": 6333},
            },
        }
        try:
            self.memory = Memory.from_config(config)
        except Exception as e:
            print(f"메모리 초기화 실패: {e}")
            self.memory = None
            
        self.client = OpenAI()
        self.app_id = "customer-support"
    
    def handle_query(self, query: str, customer_id: str) -> str:
        """
        고객 문의를 처리하고 관련 정보를 메모리에 저장한다.
        """
        if not self.memory:
            return "메모리 시스템이 초기화되지 않았다."

        # 1. 고객의 과거 기록 검색
        past_memories = self.memory.search(query=query, user_id=customer_id, limit=5)
        
        # 2. 과거 기록을 텍스트로 변환
        context = ""
        if past_memories and 'results' in past_memories:
            memory_items = [f"- {m['memory']}" for m in past_memories['results'] if isinstance(m, dict) and 'memory' in m]
            if memory_items:
                context = "고객 과거 기록:\n" + "\n".join(memory_items)
        
        # 3. 시스템 프롬프트 구성
        system_prompt = f"""당신은 전문적이고 친절한 고객 지원 AI 에이전트다.
고객의 문의를 정확하게 이해하고 도움이 되는 답변을 제공한다.

{context}
"""
        
        # 4. AI 응답 생성
        response = self.client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": query},
            ],
        )
        
        answer = response.choices[0].message.content
        
        # 5. 문의 내용을 메모리에 저장
        self.memory.add(
            query, 
            user_id=customer_id, 
            metadata={"app_id": self.app_id, "type": "customer_query"}
        )
        
        return answer
    
    def get_customer_history(self, customer_id: str) -> list:
        """
        특정 고객의 모든 메모리를 검색한다.
        """
        if not self.memory:
            return []
            
        all_memories = self.memory.get_all(user_id=customer_id)
        
        if all_memories and 'results' in all_memories:
            return all_memories['results']
        return []

# 사용 예제
print("=== 고객 지원 에이전트 데모 ===\n")

# 에이전트 초기화
support_agent = CustomerSupportAgent()
customer_id = "customer_12345"

# 첫 번째 문의
print("고객: 최근 주문에 도움이 필요합니다. 아직 도착하지 않았어요.")
response1 = support_agent.handle_query(
    "최근 주문에 도움이 필요합니다. 아직 도착하지 않았어요.", 
    customer_id=customer_id
)
print(f"에이전트: {response1}\n")

# 두 번째 문의 (다음 날)
print("고객: 어제 문의한 주문 건은 어떻게 되었나요?")
response2 = support_agent.handle_query(
    "어제 문의한 주문 건은 어떻게 되었나요?", 
    customer_id=customer_id
)
print(f"에이전트: {response2}\n")

# 고객 기록 확인
print("=== 저장된 고객 메모리 ===")
history = support_agent.get_customer_history(customer_id)
for item in history:
    if isinstance(item, dict) and 'memory' in item:
        print(f"- {item['memory']}")

=== 고객 지원 에이전트 데모 ===

고객: 최근 주문에 도움이 필요합니다. 아직 도착하지 않았어요.
에이전트: 주문이 아직 도착하지 않으셨군요. 불편을 드려 죄송합니다. 주문 번호를 알려주시면, 배송 상태를 확인하고 도와드릴 수 있습니다. 한편, 예상 배송일이나 문의하신 주문과 관련된 다른 정보도 제공해 주시면 도움이 될 것 같습니다.

고객: 어제 문의한 주문 건은 어떻게 되었나요?
에이전트: 고객님, 어제 문의하신 주문 건에 대해 확인해 보겠습니다. 주문이 아직 도착하지 않았다고 하셨는데, 주문 번호를 알려주시면 더 자세히 확인해드리겠습니다. 감사합니다!

=== 저장된 고객 메모리 ===
- 최근 주문에 도움이 필요함
- 어제 문의한 주문 건에 대한 질문
- 주문이 아직 도착하지 않음


## 5. 메모리 관리 모범 사례

### 5.1 기본 메모리 vs mem0 선택 가이드

| 상황 | 권장 방식 | 이유 |
|---|---|---|
| 짧은 대화 (5턴 이하) | 기본 메모리 | 구현이 간단하고 충분하다. |
| 장기 대화 | mem0 | 토큰 절약, 선택적 정보 저장이 가능하다. |
| 다중 세션 기억 필요 | mem0 | 영구 저장소가 필요하다. |
| 실시간 응답 중요 | 기본 메모리 | 추가 검색 단계가 없어 빠르다. |
| 사용자별 개인화 | mem0 | 사용자 ID 기반으로 관리가 용이하다. |

### 5.2 메모리 검색 최적화

In [12]:
if memory: # 메모리가 초기화되었는지 확인한다.
    # 검색 결과 제한으로 성능 향상
    memories_limited = memory.search(
        query="사용자 선호도는?",
        user_id="user_123",
        limit=3  # 상위 3개만 가져온다
    )
    print(f"제한된 검색 결과: {memories_limited}")
    
    # 메타데이터로 필터링
    # 아래 코드를 실행하려면 먼저 metadata={'category': 'orders'}로 데이터가 저장되어 있어야 한다.
    memories_filtered = memory.search(
        query="주문 관련 문의",
        user_id="customer_456",
        filters={"category": "orders"}  # 특정 카테고리만 검색한다
    )
    print(f"필터링된 검색 결과: {memories_filtered}")

제한된 검색 결과: {'results': []}
필터링된 검색 결과: {'results': []}


### 5.3 메모리 정리 및 관리

In [13]:
if memory:
    # 특정 메모리 삭제 (실제 memory_id로 대체해야 한다)
    # memory.delete(memory_id="mem_abc123")

    # 사용자의 모든 메모리 삭제
    # memory.delete_all(user_id="user_789")

    print("메모리 삭제 기능은 주석 처리되어 있다. 필요시 주석을 해제하고 사용한다.")

# 오래된 메모리 주기적으로 정리 (구현 예시)
def cleanup_old_memories(memory_instance, user_id, days=30):
    """지정된 일수보다 오래된 메모리를 삭제한다."""
    from datetime import datetime, timedelta
    
    try:
        all_memories = memory_instance.get_all(user_id=user_id)
        cutoff_date = datetime.now() - timedelta(days=days)
        
        for mem in all_memories.get('results', []):
            created_at_str = mem.get('created_at')
            if created_at_str:
                created_at = datetime.fromisoformat(created_at_str.replace('Z', '+00:00'))
                if created_at.replace(tzinfo=None) < cutoff_date:
                    print(f"오래된 메모리 삭제: {mem['id']}")
                    memory_instance.delete(mem['id'])
    except Exception as e:
        print(f"메모리 정리 중 오류 발생: {e}")


메모리 삭제 기능은 주석 처리되어 있다. 필요시 주석을 해제하고 사용한다.


### 5.4 에러 핸들링

In [14]:
def safe_memory_search(memory_instance, query, user_id):
    """안전한 메모리 검색 (에러 핸들링 포함) 함수다."""
    try:
        results = memory_instance.search(query=query, user_id=user_id, limit=5)
        
        if results and 'results' in results:
            return results['results']
        else:
            print("검색 결과가 없다.")
            return []
            
    except Exception as e:
        print(f"메모리 검색 중 오류 발생: {e}")
        return []

def safe_memory_add(memory_instance, messages, user_id):
    """안전한 메모리 추가 (에러 핸들링 포함) 함수다."""
    try:
        memory_instance.add(messages, user_id=user_id)
        print("메모리가 성공적으로 저장되었다.")
        
    except Exception as e:
        print(f"메모리 저장 중 오류 발생: {e}")

## 7.3 토큰 한계 초과 해결
- **문제**: 대화 기록이 너무 길어져 컨텍스트 창을 초과한다.
- **해결 방법**: 메시지 목록의 일부(예: 최근 10개)만 유지하도록 잘라내는 로직을 추가한다.

In [15]:
def truncate_messages(messages, max_messages=10):
    """메시지 개수를 제한한다 (시스템 메시지는 유지)."""
    system_msgs = [m for m in messages if m["role"] == "system"]
    other_msgs = [m for m in messages if m["role"] != "system"]
    
    # 최근 메시지만 유지한다
    recent_msgs = other_msgs[-max_messages:]
    
    return system_msgs + recent_msgs

# 사용 예시
long_conversation = SimpleConversation()
for i in range(15):
    long_conversation.add_user_message(f"질문 {i+1}")
    long_conversation.add_assistant_message(f"답변 {i+1}")

print(f"자르기 전 메시지 개수: {len(long_conversation.messages)}")
truncated = truncate_messages(long_conversation.messages, max_messages=10)
print(f"자른 후 메시지 개수: {len(truncated)}")

자르기 전 메시지 개수: 30
자른 후 메시지 개수: 10
